In [ ]:
# # today = (Sunny, Hot, Normal, False) -> output should be YES
from sklearn import tree
import csv
import numpy as np


data = np.array([])
target = []
featureName = []
targetName = []

In [ ]:
def readDataIntoNumpy( ):
    global data, target, featureName, targetName
    tx = []
    ty = []
    labels = []
    columnName = []
    fileName = "soccer.csv"
    indexToIgnore = [0,3,4,6]  # ignoring the useless columns
    with open(fileName,encoding="utf8") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for i, rows in enumerate(csv_reader):
#             if(i>5000):
#                 break #breaking early just for quick runs
            tempData = []
            for j,data in enumerate(rows):
                if j not in indexToIgnore:
                    tempData.append(data)
            if i>0:
                datas = tempData[:-1]
                label = (tempData[-1:])
                if(label not in labels):
                    labels.append(tempData[ len(tempData)-1 ])
                tx.append(datas)
#                 ty.append( labels.index(label) )
                ty.append(label)
                # storage.append(tempData)
            else:
                columnName.extend(tempData)
    data = tx
#     data = np.array(tx)
    target = ty
#    target = np.array(ty)
#     targetName = np.array(labels)
    targetName = labels
    featureName = columnName[:-1]
#     featureName = np.array(columnName[:-1])
    
#print(allData)

In [ ]:
readDataIntoNumpy()

# print("Data In Order: ")
# print("Target names: ")
# print(targetName)
# print("Features")
# print(featureName)
# print("Data")
# print(data)
# print("Targets")
# print(target)

In [ ]:
from sklearn import preprocessing


# changing categorical data to one hot
enc = preprocessing.OneHotEncoder()
enc.fit(data)
encData = enc.transform(data)

eTarget = preprocessing.OneHotEncoder()
eTarget.fit(target)
encTarget = eTarget.transform(target)
# print(encData.toarray())
# print(enc.categories_)
# print(eTarget.categories_)
onehotFeatureNames = []
for i, val in enumerate(enc.categories_):
#     print(featureName[i])
    for j in val:
        onehotFeatureNames.append("{}:{}".format(featureName[i],j))
print(onehotFeatureNames)

In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(encData.toarray(), encTarget.toarray())
# tree.plot_tree(clf) 
# print(encData.toarray())
# print(encTarget.toarray())

import graphviz 
# dot_data = tree.export_graphviz(clf, out_file=None) 
# graph = graphviz.Source(dot_data) 
# graph.render("rainy") 


dot_data = tree.export_graphviz(clf, out_file=None, 
            feature_names=onehotFeatureNames,  
            class_names=targetName,  
            filled=True, rounded=True,  
            special_characters=True)  
graph = graphviz.Source(dot_data) 
graph.render("soccerData")
 
# graph 

In [ ]:

#today = (Sunny, Hot, Normal, False)
testData = [ ['Wales', 'Scotland', 'Friendly', 'Wales'] ]
print(testData)
testData = enc.transform( testData )
prediction = clf.predict(testData.toarray())
pred = eTarget.inverse_transform(prediction)
print(pred[0][0])
